In [1]:
arquivo = r"C:\Users\usuario\OneDrive\Área de Trabalho\programas\OBRAS MACHADO DE ASSIS\raw\txt\romance\memoriasBras.txt"
with open(arquivo,'r',encoding='utf-8')as f:
    texto = f.read()

In [2]:
char = sorted(list(set(texto)))

In [3]:
stoi = {ch:i for i,ch in enumerate(char)}
itos = {i:ch for i,ch in enumerate(char)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
tensorlist = lambda v: [t.item() for t in v]

In [4]:
import torch
data = torch.tensor(encode(texto),dtype=torch.long)
print(data.shape, data.dtype)
print(decode(i) for i in data[:1000])

torch.Size([354375]) torch.int64
<generator object <genexpr> at 0x000001E615D91700>


In [5]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
train_data[:block_size+1]

tensor([ 36,  53,  61, 100,  66,  57,  49,  67,   1])

In [7]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs___________________________')
print(xb.shape)
print(xb)
print()
print('target___________________________')
print(yb.shape)
print(yb)
print()
print("=====================")
print()

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"quando o input é {context.tolist()} o alvo é {target}")



inputs___________________________
torch.Size([4, 8])
tensor([[  1,  52, 103,  70,  57,  52,  49,   0],
        [  1,  70,  57,  49,   1,  49,  63,   1],
        [ 66,  53,  68,  63,  66,  65,  69,  57],
        [  0,  62,  69,  62,  51,  49,   1,  58]])

target___________________________
torch.Size([4, 8])
tensor([[ 52, 103,  70,  57,  52,  49,   0,  66],
        [ 70,  57,  49,   1,  49,  63,   1,  60],
        [ 53,  68,  63,  66,  65,  69,  57,  49],
        [ 62,  69,  62,  51,  49,   1,  58,  49]])


quando o input é [1] o alvo é 52
quando o input é [1, 52] o alvo é 103
quando o input é [1, 52, 103] o alvo é 70
quando o input é [1, 52, 103, 70] o alvo é 57
quando o input é [1, 52, 103, 70, 57] o alvo é 52
quando o input é [1, 52, 103, 70, 57, 52] o alvo é 49
quando o input é [1, 52, 103, 70, 57, 52, 49] o alvo é 0
quando o input é [1, 52, 103, 70, 57, 52, 49, 0] o alvo é 66
quando o input é [1] o alvo é 70
quando o input é [1, 70] o alvo é 57
quando o input é [1, 70, 57] o alvo é 

In [8]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    
    def __int__(self, vocab_size):
        super().__int__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        return logits

m = BigramLanguageModel(vocab_size)
out = m(xb,yb)
print(out.shape)

NameError: name 'vocab_size' is not defined